In [2]:
# 모듈 load
import pandas as pd
import numpy as np
import json

import torch
from tokenization_kobert import KoBertTokenizer
from transformers import AutoModelForSequenceClassification

In [1]:
import time

In [3]:
# intent_list 생성

with open('data/동원몰 챗봇_☆상용서버☆Live-dialog.json', 'r',encoding="utf-8") as f:
    json_data = json.load(f)

intent_list = []
for x in json_data["intents"]:
    intent_list.append(x["intent"])

In [4]:
# dialogue 읽기

with open('data/dialogue_test.json', 'r',encoding="utf-8") as f2:
    json_data2 = json.load(f2)

In [5]:
# gpu 메모리 청소
torch.cuda.empty_cache()

# device = gpu 설정
if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('%d개 존재함' % torch.cuda.device_count())
    print('사용할 GPU : ', torch.cuda.get_device_name(0))

# If not...
else:
    print('GPU 없어서 CPU로 설정')
    device = torch.device("cpu")

1개 존재함
사용할 GPU :  NVIDIA GeForce RTX 3060


In [6]:
# 모델 load -> (나스 개인 폴더에 있으니 README.MD 참조해서 가져오셈
model2 = AutoModelForSequenceClassification.from_pretrained("/model/kobert").cuda()

# 토크나이저 선언
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

# load 파라미터(변경시 꼭 수정해야해!)
max_len = 152

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [8]:
# 모델 돌려돌려
def MRC(input_ids,attention_masks,start):
    model2.eval()

    predictions = []
    last_layer_attentions = []

    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    for i in range(len(input_ids)):
        ids = input_ids[i].unsqueeze(0)
        masks = attention_masks[i].unsqueeze(0)

        with torch.no_grad():
            outputs = model2(input_ids=ids,
                            attention_mask=masks)

        logits = outputs[0]
        logits = torch.softmax(logits,dim=1)
        last_layer_attention = outputs[1][-1]

        logits = logits.detach().cpu().numpy()
        last_layer_attention = last_layer_attention.detach().cpu().numpy()

        last_layer_attentions.append(last_layer_attention) 
        predictions.append(logits)

    probs = predictions[0][0]
    pred_idx = np.argmax(probs)

    return intent_list[pred_idx]

In [9]:
# intents에 해당하는 다이얼로그 불러오고 상태 출력

def DIA(start2):
    if start2 == "처음으로":
        time.sleep(0.5)
        print("다시 돌아가겠습니다.")
        ending_temp = []
        return ending_temp
        
    for x in json_data2: # 
        if x["title"] == start2:
            ending_temp = [x["output"]["text"]["values"],x["next_step"]]
            
            if x["type"] == "하나 선택":
                ending_temp.append(x["output"]["table"])
                return ending_temp
            
            return ending_temp
            ## elif 의 무수한 type 경우수
# 이것도 저것도 아니면?

In [10]:
# intents에 해당하는 슬롯 불러오기


In [11]:
### 시작 ###
okay = 0
text = "안녕하세요 동원 챗봇 푸디입니다. 언제든 말씀해 주세요."
while True:
    # 처음
    if okay == 0:
        start = input(text)

        # 종료시
        if start == "종료":
            break

        # 토큰화 하기
        input_ids = []
        attention_masks = []

        ## 읽고 토큰화하기
        encoded_dict = tokenizer.encode_plus(
                                start,            
                                add_special_tokens = True,
                                max_length = max_len,
                                pad_to_max_length = True, # padding =True 나 padding="longest" 같은걸로 대체하세요
                                return_attention_mask = True,
                                return_tensors = 'pt')
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        # mrc 돌리기
        start2 = MRC(input_ids,attention_masks,start)
        okay = 1 # 2번쨰 전환
        ending = DIA(start2) # [문장,다음스텝,테이블]

        if ending == None:
            time.sleep(0.5)
            print("이해하기 어려워요. 쉽게 얘기해주세요")
            okay = 0
            continue
        
        # 테이블이 존재하면 출력해주기
        if len(ending) == 3:
            time.sleep(0.5)
            print(str(ending[2]))
        
    # 2번째부터
    if okay == 1:
        if ending == None:
            time.sleep(0.5)
            print("이해하기 어려워요. 쉽게 얘기해주세요")
            okay = 0
            continue
        start = input(ending[0])

        if start == "종료":
            break

        ## ending : values, next_step, (table)

        ## 테이블이 있는 경우,
        if len(ending) == 3:
            if start in ending[2]: #테이블에 작성한 답이 있다면?
                ending = DIA(ending[1][0]) # 정답
                time.sleep(0.5)
                print(str(ending[0][0])) # 랜덤없이 value 0번쨰 출력해줌
            else: #테이블에 작성한 답이 없다면?
                ending = DIA(ending[1][1]) # 오답
                time.sleep(0.5)
                print(str(ending[0][0])) # 랜덤없이 value 0번쨰 출력해줌
                
        ## 테이블이 없는 경우,
        elif len(ending) == 2:
            # next step이 없는경우
            if len(ending[1]) == 0:
                okay = 0
            # next step이 3개 이상이면,(아직 미구현)
            elif len(ending[1]) > 2:
                ## 추후에 intents 정리되면 mrc2(긍정,부정, 1번, 2번..)로 처리 하면 되는 구역
                if MRC(start) == "긍정":
                    ending = DIA(ending[1][0])

                elif MRC(start) == "부정":
                    ending = DIA(ending[1][1])
                else:
                    ending = DIA("몰루")
        # 문장만 있는경우 = 미구현(지워도 무방)
        elif len(ending) == 1:
            ending = DIA(ending[1][0])

        # 아예 없다 => 다시 처음으로 돌리기 위해 만듬
        elif len(ending) == 0:
            okay = 0


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\YU\miniconda3\envs\py38\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


이해하기 어려워요. 쉽게 얘기해주세요
이해하기 어려워요. 쉽게 얘기해주세요
['사과', '바나나', '초코파이']
확인했습니다. 해당 상품의 배송 조회 페이지로 이동하겠습니다. --페이지--
이해하기 어려워요. 쉽게 얘기해주세요
